In [115]:
import os
import sys
import subprocess
import argparse
import logging
import plotly.graph_objects as go
import pysam
from tqdm import tqdm
import numpy as np

In [116]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


work_dir is the directory of working folder

In [117]:
with open("config", "r") as config_f:
    parameters = config_f.readlines()

In [118]:
threads = int(parameters[0])
percentile = int(parameters[1])
work_dir = parameters[2][:-1]
ngmlr_above_bam = parameters[3][:-1]
ngmlr_above_sam = parameters[4][:-1]
ngmlr_above_edit_distance = parameters[5][:-1]
minimap2_above_bam = parameters[6][:-1]
minimap2_above_sam = parameters[7][:-1]
minimap2_above_edit_distance = parameters[8][:-1]
minimap2_full_bam = parameters[9][:-1]
minimap2_full_edit_distance = parameters[10][:-1]
final_bam = parameters[11][:-1]
final_sam = parameters[12][:-1]
final_edit_distance = parameters[13][:-1]
raw_edit_distance = bool(int(parameters[14]))

In [119]:
def get_distance_from_file(distance_file):
    with open(distance_file, "r") as distance_f:
        distance = distance_f.readlines()
    distance_list = []
    for dist in distance:
        distance_list.append(float(dist[:-1].split(":")[2]))
    return distance_list

In [120]:
def edit_distance_compare(ngmlr_distance_file, minimap2_distance_file, raw_edit_distance, hybrid):
#     ngmlr_distance.sort()
#     minimap2_distance.sort()
    fig = go.Figure()

    ngmlr_distance = get_distance_from_file(ngmlr_distance_file)
    minimap2_distance = get_distance_from_file(minimap2_distance_file)
    lendic_minimap2 = {}
    lencnts_minimap2=[]
    if raw_edit_distance:
        for i in minimap2_distance:
            temploc = int(i/100)*100
            if temploc not in lendic_minimap2:
                lendic_minimap2.update({temploc:0})
            else:
                lendic_minimap2[temploc] += 1
        lens_minimap2 =  [str(i) for i in np.array(list(sorted(lendic_minimap2.keys())))]
        for i in sorted(lendic_minimap2.keys()):
            lencnts_minimap2.append(lendic_minimap2[i])


        lendic_ngmlr = {}
        lencnts_ngmlr = []
        for i in ngmlr_distance:
            temploc = int(i/100)*100
            if temploc not in lendic_ngmlr:
                lendic_ngmlr.update({temploc:0})
            else:
                lendic_ngmlr[temploc] += 1

        lens_ngmlr =  [str(i) for i in np.array(list(sorted(lendic_ngmlr.keys())))]
        for i in sorted(lendic_ngmlr.keys()):
            lencnts_ngmlr.append(lendic_ngmlr[i])
        fig.update_layout(
            title="Raw edit distance distribution",
            xaxis_title="edit distances",
            yaxis_title="Numbers",
            )
    else:
        for i in minimap2_distance:
            temploc = int(i/0.005)*0.005
            if temploc not in lendic_minimap2:
                lendic_minimap2.update({temploc:0})
            else:
                lendic_minimap2[temploc] += 1
        lens_minimap2 =  [str(i) for i in np.array(list(sorted(lendic_minimap2.keys())))]
        for i in sorted(lendic_minimap2.keys()):
            lencnts_minimap2.append(lendic_minimap2[i])

        lendic_ngmlr = {}
        lencnts_ngmlr = []
        for i in ngmlr_distance:
            temploc = int(i/0.005)*0.005
            if temploc not in lendic_ngmlr:
                lendic_ngmlr.update({temploc:0})
            else:
                lendic_ngmlr[temploc] += 1

        lens_ngmlr =  [str(i) for i in np.array(list(sorted(lendic_ngmlr.keys())))]
        for i in sorted(lendic_ngmlr.keys()):
            lencnts_ngmlr.append(lendic_ngmlr[i])
        fig.update_layout(
            title="Normalized edit distance distribution",
            xaxis_title="edit distances",
            yaxis_title="Numbers",
            )
    fig.add_trace(go.Bar(x= lens_minimap2, y=lencnts_minimap2,  name='minimap2'))
    fig.add_trace(go.Scatter(x= lens_minimap2, y=lencnts_minimap2,  name='minimap2'))
    if hybrid:
        fig.add_trace(go.Bar(x= lens_ngmlr, y=lencnts_ngmlr, name='hybrid_NGMLR'))
        fig.add_trace(go.Scatter(x= lens_ngmlr, y=lencnts_ngmlr, name='hybrid_NGMLR'))
    else:
        fig.add_trace(go.Bar(x= lens_ngmlr, y=lencnts_ngmlr, name='NGMLR'))
        fig.add_trace(go.Scatter(x= lens_ngmlr, y=lencnts_ngmlr, name='NGMLR'))

    fig.show()

In [121]:
def bam_to_sam(input_bam, output_sam):
    transform_cmd = f"samtools view -h -@ {threads} {input_bam} -o {output_sam}"
    logger.info(transform_cmd)
    os.system(transform_cmd)

In [122]:
def generate_distance_file(input_sam, output_txt, raw_edit_distance):
    if raw_edit_distance:
        os.system(f"grep -o -E \"NM:i:[0-9]+\" {input_sam} > {output_txt}")
    else:
        samfile = pysam.AlignmentFile(input_sam, "r")
        with open(output_txt, "w") as out_f:
            for read in tqdm(samfile.fetch()):
                tags = dict(read.tags)
                if "NM" in tags:
                    NM_distance = int(tags["NM"])
                    normalized_edit_distance = float(
                        NM_distance)/read.query_alignment_length
                    out_f.writelines(f"NM:i:{normalized_edit_distance}\n")
    return output_txt

In [123]:
bam_to_sam(ngmlr_above_bam, ngmlr_above_sam)

2020-10-20 14:12:20,868 - __main__ - INFO - samtools view -h -@ 60 ../Saccharomyces_cerevisiae/work/ngmlr_above85.bam -o ../Saccharomyces_cerevisiae/work/ngmlr_above85.sam


In [124]:
bam_to_sam(minimap2_above_bam, minimap2_above_sam)

2020-10-20 14:12:21,659 - __main__ - INFO - samtools view -h -@ 60 ../Saccharomyces_cerevisiae/work/minimap2_above85.bam -o ../Saccharomyces_cerevisiae/work/minimap2_above85.sam


In [125]:
generate_distance_file(ngmlr_above_sam, ngmlr_above_edit_distance, raw_edit_distance)

10554it [00:00, 44495.21it/s]


'../Saccharomyces_cerevisiae/work/ngmlr_above85_distance.txt'

In [126]:
generate_distance_file(minimap2_above_sam, minimap2_above_edit_distance, raw_edit_distance)

10675it [00:00, 50526.71it/s]


'../Saccharomyces_cerevisiae/work/minimap2_above85_distance.txt'

In [127]:
edit_distance_compare(ngmlr_above_edit_distance, minimap2_above_edit_distance, raw_edit_distance, hybrid=False)

In [128]:
bam_to_sam(final_bam, final_sam)

2020-10-20 14:12:23,871 - __main__ - INFO - samtools view -h -@ 60 ../Saccharomyces_cerevisiae/work/hybrid_ngmlr.bam -o ../Saccharomyces_cerevisiae/work/final.sam


In [129]:
generate_distance_file(final_sam, final_edit_distance, raw_edit_distance)

65713it [00:01, 41721.91it/s]


'../Saccharomyces_cerevisiae/work/final_edit_distance.txt'

In [130]:
edit_distance_compare(final_edit_distance, minimap2_full_edit_distance, raw_edit_distance, hybrid=True)

In [131]:
def sv_calling_evaluation(min_read, final_bam, minimap2_bam, simulated_sv_bed, bp_shift):
    final_sv = os.path.join(work_dir, "final_sniffles.vcf")
    final_sv_eval = os.path.join(work_dir, "final_eval")
    minimap2_sv = os.path.join(work_dir, "minimap2_full_sniffles.vcf")
    minimap2_sv_eval = os.path.join(work_dir, "minimap2_full_eval")
    
    hybrid_NGMLR_sv_cmd = f"sniffles -s {min_read} -m {final_bam} -v {final_sv} -t {threads}"
    minimap2_sv_cmd = f"sniffles -s {min_read} -m {minimap2_bam} -v {minimap2_sv} -t {threads}"
    hybrid_NGMLR_sv_eval_cmd = f"SURVIVOR eval {final_sv} {simulated_sv_bed} {bp_shift} {final_sv_eval} > {final_sv_eval}.out"
    minimap2_sv_eval_cmd = f"SURVIVOR eval {minimap2_sv} {simulated_sv_bed} {bp_shift} {minimap2_sv_eval} > {minimap2_sv_eval}.out"
    
    os.system(hybrid_NGMLR_sv_cmd)
    os.system(minimap2_sv_cmd)
    os.system(hybrid_NGMLR_sv_eval_cmd)
    os.system(minimap2_sv_eval_cmd)
    
    with open(f"{final_sv_eval}.out", "r") as eval_file:
        print(eval_file.readlines()[-1])
    with open(f"{minimap2_sv_eval}.out", "r") as eval_file:
        print(eval_file.readlines()[-1])

In [132]:
simulated_sv_bed = "/home/Users/yf20/projects/hybrid_NGMLR/Saccharomyces_cerevisiae/sv_simulated.bed"

In [134]:
sv_calling_evaluation(2, final_bam, minimap2_full_bam, simulated_sv_bed, 30)

 Overall: 145 32/24/38/40/4 4/1/2/0/0 24/7/4/3/11 0.951724 0.262032

 Overall: 145 30/10/34/40/4 6/15/6/0/0 27/7/9/2/28 0.813793 0.382199



In [135]:
def sv_calling_evaluation_dry(min_read, final_bam, minimap2_bam, simulated_sv_bed, bp_shift):
    final_sv = os.path.join(work_dir, "final")
    final_sv_eval = os.path.join(work_dir, "final_eval")
    minimap2_sv = os.path.join(work_dir, "minimap2_full")
    minimap2_sv_eval = os.path.join(work_dir, "minimap2_full_eval")
    
    hybrid_NGMLR_sv_cmd = f"sniffles -s {min_read} -m {final_bam} -v {final_sv} -t {threads}"
    minimap2_sv_cmd = f"sniffles -s {min_read} -m {minimap2_bam} -v {minimap2_sv} -t {threads}"
    hybrid_NGMLR_sv_eval_cmd = f"SURVIVOR eval {final_sv} {simulated_sv_bed} {bp_shift} {final_sv_eval} > {final_sv_eval}.out"
    minimap2_sv_eval_cmd = f"SURVIVOR eval {minimap2_sv} {simulated_sv_bed} {bp_shift} {minimap2_sv_eval} > {minimap2_sv_eval}.out"
    
    print(hybrid_NGMLR_sv_cmd)
    print(minimap2_sv_cmd)
    print(hybrid_NGMLR_sv_eval_cmd)
    print(minimap2_sv_eval_cmd)
    
    with open(f"{final_sv_eval}.out", "r") as eval_file:
        print(eval_file.readlines()[-1])
    with open(f"{minimap2_sv_eval}.out", "r") as eval_file:
        print(eval_file.readlines()[-1])

In [136]:
sv_calling_evaluation_dry(2, final_bam, minimap2_full_bam, simulated_sv_bed, 30)

sniffles -s 2 -m ../Saccharomyces_cerevisiae/work/hybrid_ngmlr.bam -v ../Saccharomyces_cerevisiae/work/final -t 60
sniffles -s 2 -m ../Saccharomyces_cerevisiae/work/minimap2_full.bam -v ../Saccharomyces_cerevisiae/work/minimap2_full -t 60
SURVIVOR eval ../Saccharomyces_cerevisiae/work/final /home/Users/yf20/projects/hybrid_NGMLR/Saccharomyces_cerevisiae/sv_simulated.bed 30 ../Saccharomyces_cerevisiae/work/final_eval > ../Saccharomyces_cerevisiae/work/final_eval.out
SURVIVOR eval ../Saccharomyces_cerevisiae/work/minimap2_full /home/Users/yf20/projects/hybrid_NGMLR/Saccharomyces_cerevisiae/sv_simulated.bed 30 ../Saccharomyces_cerevisiae/work/minimap2_full_eval > ../Saccharomyces_cerevisiae/work/minimap2_full_eval.out
 Overall: 145 32/24/38/40/4 4/1/2/0/0 24/7/4/3/11 0.951724 0.262032

 Overall: 145 30/10/34/40/4 6/15/6/0/0 27/7/9/2/28 0.813793 0.382199

